### Question 1 - SQL

### Part A

You have a table populated with trip information (named uber_trip) table with a 

rider_id (unique per rider)

, trip_id (unique per trip)

, trip_timestamp_utc (the UTC timestamp for when the trip began)

, and trip_status, which can either be ‘completed’ or ‘not completed’.

rider_id , trip_id, begintrip_timestamp_utc, trip_status

Write a query to return the trip_id for the 5th completed trip for each rider. If a rider has
completed fewer than five trips, then don’t include them in the results.


### Part A solution 

    with ranked_table as

    (

    select *, rank () over (partition by rider_id order by trip_timestamp_utc ASC) as rank_triptime

    from uber_trip

    where trip_status = 'completed'

    )


    select rider_id, trip_id 

    from ranked_table
    where rank_triptime = 5


### Part B

You are given three separate tables (named trip_initiated, trip_cancel, and trip_complete) of the form:

trip_initiated | trip_id, rider_id, driver_id, timestamp

trip_cancel | trip_id, rider_id, driver_id, timestamp

trip_complete | trip_id, rider_id, driver_id, timestamp

Each trip_id in these tables will be unique and only appear once, and a trip will only ever result in
a single cancel event or it will be completed. Write a query to create a single table with one row per trip event sequence (trip initiated → cancel/complete):

dispatch_events | trip_id, rider_id, driver_id, initiated_ts, cancel_ts, complete_ts

There should only be a single row per trip with a unique trip_id.


### Part B solution


    with dispatch_events as 

    (

        select t1.trip_id, t1.rider_id, t1.driver_id
        , t1.timestamp AS initiated_ts
        , t2.timestamp AS cancel_ts
        , t3.timestamp AS complete_ts

        from trip_initiated t1 

        left join trip_cancelled t2 on t1.trip_id = t2.trip_id

        left join trip_complete t3 on t1.trip_id = t3.trip_id

    )



### Part C

Write at least one test query to validate the data in the resulting table.

Indicate what you would expect the query to return if the data were valid

### Part C solution

some sanity checks:

queries will return 'pass' if expected output is valid

--------------------------------------------------------------------------------------------------------

-number of trips in trip_initiated is same as in dispatched_events


    select concat( 'sanity check on # trip_id: ', 

    case when

    (select count(trip_id) from trip_initiated) = (select count(trip_id) from dispatched_events)

    then 'passed' else 'failed' end);

--------------------------------------------------------------------------------------------------------

-number of trips in trip_cancelled is same as number of trips in dispatch events with complete_ts = null values
-number of trips in trip_complete is same as number of trips in dispatch events with cancel_ts = null values

    select concat( 'sanity check on timestamp null values ', 

    case when

    (select count(trip_id) from trip_cancelled) = (select count(trip_id) from dispatched_events where complete_ts is null)

    and (select count(trip_id) from trip_complete) = (select count(trip_id) from dispatched_events where cancel_ts is null)

    then 'passed' else 'failed' end);






